In [ ]:
~/backup_git
~/git
sudo tar -czvf /mnt/backup/epinyoan/git_no_momask.tar.gz --exclude=./momask-codes ./

sudo tar -xvzf git_no_momask.tar.gz -C git_no_momask
screen -r 59691

In [1]:
import sys
sys.path.append('../')
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"]="7"

In [2]:
import os
from os.path import join as pjoin

import torch
from torch.utils.data import DataLoader

from models.vq.model import RVQVAE
from models.vq.vq_trainer import RVQTokenizerTrainer
from options.vq_option import arg_parse
from data.t2m_dataset import MotionDataset
from utils import paramUtil
import numpy as np

from models.t2m_eval_wrapper import EvaluatorModelWrapper
from utils.get_opt import get_opt
from motion_loaders.dataset_motion_loader import get_dataset_motion_loader

from utils.motion_process import recover_from_ric
from utils.plot_script import plot_3d_motion
import datetime
from data.t2m_dataset import Text2MotionDataset

/home/epinyoan/miniconda3/envs/momask/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Temp:
    def __init__(self):
        print('mock:: opt')
opt = Temp()
opt.data_root = './dataset/HumanML3D/'
opt.motion_dir = pjoin(opt.data_root, 'new_joint_vecs')
opt.text_dir = pjoin(opt.data_root, 'texts')
opt.joints_num = 22
opt.is_train = False # So it won't overwrite mean/std
dim_pose = 263
fps = 20
radius = 4
kinematic_chain = paramUtil.t2m_kinematic_chain
dataset_opt_path = './checkpoints/t2m/Comp_v6_KLD005/opt.txt'

mock:: opt


In [7]:
mean = np.load('./dataset/HumanML3D/Mean.npy')
std = np.load('./dataset/HumanML3D/Std.npy')
### First stage
opt.feat_bias = 5
train_split_file = pjoin(opt.data_root, '/home/epinyoan/git/momask-codes/dataset/HumanML3D/val.txt')
opt.window_size = 64
opt.batch_size = 512
train_dataset = MotionDataset(opt, mean, std, train_split_file)
train_loader = DataLoader(train_dataset, batch_size=opt.batch_size, num_workers=4, shuffle=True, drop_last=True)

  0%|          | 0/1460 [00:00<?, ?it/s]

100%|██████████| 1460/1460 [00:00<00:00, 2510.83it/s]

Total number of motions 1300, snippets 116698


In [14]:
opt.nb_code = 512 # doesn't matter
opt.code_dim = 512
opt.down_t = 2
opt.stride_t = 2
opt.width = 512
opt.depth = 3
opt.dilation_growth_rate = 3
opt.vq_act = 'relu'
opt.vq_norm = None

opt.num_quantizers = 6
opt.shared_codebook = False
opt.quantize_dropout_prob = 0.2
opt.mu = .99

# dataset
opt.dataset_name = 't2m'

# vqdir = '/home/epinyoan/git/momask-codes/checkpoints/t2m/2023-12-21-16-46-46_lfq_cdim15_ent.1_div1_cmmt1_6e-5/'
vqdir = '/home/epinyoan/git/momask-codes/log/vq/t2m/2024-01-08-13-41-16_momask_rerun_lr2e-4_bestFID'
opt.meta_dir = vqdir+'/meta'
net = RVQVAE(opt,
            dim_pose,
            opt.nb_code,
            opt.code_dim,
            opt.code_dim,
            opt.down_t,
            opt.stride_t,
            opt.width,
            opt.depth,
            opt.dilation_growth_rate,
            opt.vq_act,
            opt.vq_norm)
net.moment = {'mean': torch.from_numpy(train_loader.dataset.mean).cuda().float(), 
                'std': torch.from_numpy(train_loader.dataset.std).cuda().float()}


# cb usage: 60309
opt.resume_pth = vqdir+'/model/latest.tar'
ckpt = torch.load(opt.resume_pth, map_location='cpu')
net.load_state_dict(ckpt['vq_model'], strict=True)
net.eval()
net.cuda()
''

''

In [48]:
net.train()
for i, batch in enumerate(train_loader):
    motion = batch
    # conds, motion, m_lens = batch
    motion = motion.detach().float().cuda()
    x_out, commit_loss, perplexity = net(motion[:2])
    break

x_out: torch.Size([2, 512, 16])


### Codebook Usage

In [6]:
### Second stage
# opt.max_motion_length = 196
# opt.batch_size = 64
# train_split_file = pjoin(opt.data_root, '/home/epinyoan/git/momask-codes/dataset/HumanML3D/train.txt')
# train_dataset = Text2MotionDataset(opt, mean, std, train_split_file)
# train_loader = DataLoader(train_dataset, batch_size=opt.batch_size, num_workers=4, shuffle=True, drop_last=True)

In [7]:
opt.unit_length = 4
net.get_codebook_usage(train_loader, stage=1)
# 25945  /32768

28521

In [49]:
all_codes = []
for i, batch in enumerate(train_loader):
    motion = batch
    # conds, motion, m_lens = batch
    motion = motion.detach().float().cuda()
    code_idx, _ = net.encode(motion)
    all_codes.append(code_idx.reshape(-1))
all_codes = torch.cat(all_codes)
u_codes, counts = all_codes.unique(return_counts=True)
# u_codes


In [50]:
counts.sort(descending=True).values[:2**14].float().std()

tensor(105.4770, device='cuda:0')

### Eval

In [19]:
from motion_loaders.dataset_motion_loader import get_dataset_motion_loader
from utils.eval_t2m import evaluation_vqvae

In [18]:
eval_val_loader, _ = get_dataset_motion_loader(dataset_opt_path, 32, 'test', device='cuda')

Reading ./checkpoints/t2m/Comp_v6_KLD005/opt.txt
Loading dataset t2m ...


  0%|          | 0/4384 [00:00<?, ?it/s]

100%|██████████| 4384/4384 [00:02<00:00, 2108.72it/s]


Pointer Pointing at 0
Ground Truth Dataset Loading Completed!!!
